# Pneumonia Detecion with ResNet50

## Dataset

In [9]:
import torch
# data handling
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset
class ChestXrayDataset(Dataset):
    def __init__(self, df:pd.DataFrame, transform=None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['path']
        image = torch.load(img_path)  # Assuming images are saved as tensors
        label = 1 if 'pneumonia' in self.image_files[idx] else 0
        
        if self.transform:
            image = self.transform(image)
        
        return image, label
    


In [10]:
# Organize data paths and labels
import os
import pandas as pd
from sklearn.model_selection import train_test_split

def resolve_data_path(data_dir: str):
    normal_dir = os.path.join(data_dir, 'NORMAL')
    pneumonia_dir = os.path.join(data_dir, 'PNEUMONIA')

    normal_paths = [os.path.join(data_dir, 'NORMAL', f) for f in os.listdir(normal_dir) if f.endswith(('.jpeg', '.jpg', '.png'))]
    pneumonia_paths = [os.path.join(data_dir, 'PNEUMONIA', f) for f in os.listdir(pneumonia_dir) if f.endswith(('.jpeg', '.jpg', '.png'))]
    pathes = normal_paths + pneumonia_paths
    labels = [0]*len(normal_paths) + [1]*len(pneumonia_paths)
    return pd.DataFrame({'path': pathes, 'label': labels})

# Base dir
train_df = resolve_data_path("datasets/all_data")
test_df = resolve_data_path("datasets/test")

train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=42)

# compute class weights
class_counts = train_df['label'].value_counts().to_dict()
total_samples = len(train_df)
class_weights = {cls: total_samples/count for cls, count in class_counts.items()}
weights = [class_weights[label] for label in train_df['label']]
# ouptput class weights
print("Class Weights:", class_weights)

print(train_df.head(20))

Class Weights: {1: 1.3473921442369607, 0: 3.8785912882298423}
                                                   path  label
3566  datasets/all_data/PNEUMONIA/person534_virus_10...      1
2866  datasets/all_data/PNEUMONIA/person940_bacteria...      1
2681  datasets/all_data/PNEUMONIA/person1322_bacteri...      1
1199         datasets/all_data/NORMAL/IM-0166-0001.jpeg      0
4619  datasets/all_data/PNEUMONIA/person464_virus_95...      1
1008  datasets/all_data/NORMAL/NORMAL2-IM-1281-0001....      0
1295  datasets/all_data/NORMAL/NORMAL2-IM-0744-0001....      0
2688  datasets/all_data/PNEUMONIA/person300_bacteria...      1
3142  datasets/all_data/PNEUMONIA/person1332_virus_2...      1
82    datasets/all_data/NORMAL/NORMAL2-IM-0577-0001....      0
4042  datasets/all_data/PNEUMONIA/person70_bacteria_...      1
5037  datasets/all_data/PNEUMONIA/person612_bacteria...      1
1980  datasets/all_data/PNEUMONIA/person273_virus_56...      1
4688  datasets/all_data/PNEUMONIA/person305_bacteria... 

In [18]:
# compute the mean and std of images in the training set
import cv2
import tqdm
import numpy as np

def compute_mean_std(df: pd.DataFrame):
    sum_ = 0.0
    sum_squared = 0.0
    num_pixels = 0

    for idx in tqdm.tqdm(range(len(df))[:10]):
        img_path = df.iloc[idx]['path']
        image = cv2.imread(img_path)
        image = image.astype('float32') / 255.0  # Normalize to [0, 1]
        # compute sum and sum of squares for each channel
        sum_ += image.sum(axis=(0, 1))
        sum_squared += (image ** 2).sum(axis=(0, 1))
        num_pixels += image.shape[0] * image.shape[1]

    mean = sum_ / num_pixels
    std = np.sqrt((sum_squared / num_pixels) - (mean ** 2))
    return mean, std
mean, std = compute_mean_std(train_df)
print(f"Mean: {mean}, Std: {std}")

100%|██████████| 10/10 [00:00<00:00, 27.49it/s]

Mean: [0.46655865 0.46655865 0.46655865], Std: [0.24664197 0.24664197 0.24664197]


In [19]:
# Define transforms, dataloaders
from torchvision import transforms

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    # Normalize with ImageNet stats, beacause resnet is pretrained on ImageNet
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

## Model Training With Cross Validation